<h1>Roomba Coverage</h1> 

A Roomba is a robotic vacuum cleaner that automatically moves
around a room or a house to clean the floor. Model the floor of a house on an integer grid
(so discrete rather than continuous) and mark cells as open floor or an obstacle (wall or
furniture).

Search for information on the Roomba and compare the effectiveness of different
movement strategies or combinations of strategies — for example, "random bounce" and
"wall following". The goal is to clean the whole floor as evenly as possible — if it misses
part of the floor that part will not get cleaned, and if it visits some floor region very often, it is
wasting time and battery power that it could be using to clean another part of the floor.

Use your simulation to evaluate different movement strategies and their effectiveness at
covering floor areas with obstacles on them.

In [2]:
#import libraries
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import copy
sns.set()
%matplotlib inline

In [213]:
#Roomba Simulation base class
class RoombaSim:

    def __init__(self, size=6, obstacle_density=0.1, strat = 'random_bounce', demo = False):
        '''
        Create a new roomba simulation object.

        Inputs:

            size (int) The grid will have dimensions of (size - 1) x (size - 1). 
                The outermost rows and cols are set as walls (fixed obstacles).
                Default: 6.

            obstacle_density (float) The fraction of cells that have an obstacle on them.
                Default: 0.3.
                
            strat (str) The strategy to be used for floor cleaning.
                Default: random_bounce.
              
            demo (bool) If True, will display the floor grid with obstacles, dust, and roomba.
                Default: False
        '''
        self.demo = demo
        self.size = size
        self.obstacle_density = obstacle_density
        self.strat = strat

        # Track the time steps.
        self.time_step = 0
        
        #set state by creating the floor
        self.state = self.create_floor()
        
        #get obstacle locations
        self.obstacle_loc = [tuple(i) for i in np.array(np.where(self.state == 1)).T]
        
        #place roomba on floor
        self.roomba_loc = self.get_start_loc(self.state)
        self.state[self.roomba_loc[0], self.roomba_loc[1]] = 8
        
        #set previous location attribute
        self.prev_loc = self.roomba_loc
        
    def create_floor(self):
        '''
        Initializes floor according to grid size and obstacle density.
        Walls are represented as obstacle as well, and occupy the outermost border of the floor grid, e.g.
        first and last row, first and last column.
        
        Outputs:
            state: numpy array of dimensions size x size representing the floor grid.
        '''
            
        #number of obstacles to place
        obstacle_count = int(self.obstacle_density*self.size*self.size)

        #initialize empty array for storing tuples of x,y values
        random_indices = np.zeros(obstacle_count, dtype=object)

        #assign obstacle_count number of random x,y values
        for i in range(obstacle_count):
            #randomly draw x,y values from range of grid size
            indices = np.random.choice(range(self.size), size=2) 
            random_indices[i] = indices[0], indices[1]
        
        # -1 means empty cell
        state = -np.ones((self.size,self.size), dtype=int) 
        
        #place obstacles on the floor
        for i in random_indices: 
            state[i[0],i[1]] = 1 #1 means obstacle
            
        #create wall
        state[0], state[-1], state[:,0], state[:,-1] = 1, 1, 1, 1
    
        return state
    
    def get_start_loc(self, state):
        '''
        Get roomba's starting location by choosing randomly from any empty space on the floor grid.
        '''
        empty_cells = np.array(np.where(state == -1)).T
        start_loc = empty_cells[np.random.choice(len(empty_cells))]

        return tuple(start_loc)
    
    def choose_pos(self):
        '''
        Choose the roomba's next position.
        '''
        #roomba's range of movement
        right = (self.roomba_loc[0], (self.roomba_loc[1] + 1))
        left = (self.roomba_loc[0], (self.roomba_loc[1] - 1))
        up = ((self.roomba_loc[0] - 1), self.roomba_loc[1])
        down = ((self.roomba_loc[0] + 1), self.roomba_loc[1])   
        pos = [right,left,up,down]
     
        #the free positions are the roomba's physical range - obstacles
        free_pos = set(pos) - set(self.obstacle_loc)
        
        if self.strat == 'random_bounce':
            #if the old position is in the set of free positions and the set has more than one element
            if self.prev_loc in free_pos and len(free_pos) > 1:
                #remove the old position to encourage the roomba to explore new areas
                free_pos.remove(self.prev_loc) 
            print(f"Possible next moves: {free_pos}")

            new_pos = list(free_pos)[np.random.choice(len(free_pos))]        
            
        return new_pos
    
    def move_roomba(self, new_pos):
        '''
        Move roomba to the given next position.
        '''
        #create a copy of the old state
        new_state = -np.ones((self.size,self.size), dtype=int) 
        for i in self.obstacle_loc: new_state[i] = 1
        
        #make the old roomba position empty again
        new_state[self.roomba_loc] = -1

        #move roomba to the next pos
        new_state[new_pos] = 8

        return new_state
        

    def step(self):
        '''
        Advance one time step in the simulation.
        '''   
        
        #if demo == True, display the roomba sim
        if self.demo: self.display()
            
        print(f"Roomba is currently at {self.roomba_loc}")
        #choose roomba's next position
        next_pos = self.choose_pos()
            
        #move roomba
        print(f"Roomba is moving to {next_pos}")
        self.state = self.move_roomba(next_pos)
        
        #set the previous position
        self.prev_loc = self.roomba_loc
        
        #set the new roomba location
        self.roomba_loc = next_pos

    def display(self):
        '''
        Print out the current state of the simulation.
        ''' 
        for row in range(self.size):
            disp = ''
            for cell in self.state[row]:
                if cell == -1: disp += '.   ' #empty spot
                elif cell == 1: disp += 'X   ' #obstacle
                else: disp += str(cell) + '   ' #roomba
            print(disp)   
        print()
        

In [214]:
np.random.seed(252)

roomba = RoombaSim(size=6,obstacle_density=0.2,demo=True)
for i in range(10):
    print(f"Step {i+1}")
    roomba.step()
    print()

Step 1
X   X   X   X   X   X   
X   .   .   .   .   X   
X   8   .   X   .   X   
X   .   .   .   .   X   
X   .   .   X   .   X   
X   X   X   X   X   X   

Roomba is currently at (2, 1)
Move directions: Right, Left, Up, Down
[(2, 2), (2, 0), (1, 1), (3, 1)]
Possible next moves: {(3, 1), (1, 1), (2, 2)}
Roomba is moving to (2, 2)

Step 2
X   X   X   X   X   X   
X   .   .   .   .   X   
X   .   8   X   .   X   
X   .   .   .   .   X   
X   .   .   X   .   X   
X   X   X   X   X   X   

Roomba is currently at (2, 2)
Move directions: Right, Left, Up, Down
[(2, 3), (2, 1), (1, 2), (3, 2)]
Possible next moves: {(1, 2), (3, 2)}
Roomba is moving to (1, 2)

Step 3
X   X   X   X   X   X   
X   .   8   .   .   X   
X   .   .   X   .   X   
X   .   .   .   .   X   
X   .   .   X   .   X   
X   X   X   X   X   X   

Roomba is currently at (1, 2)
Move directions: Right, Left, Up, Down
[(1, 3), (1, 1), (0, 2), (2, 2)]
Possible next moves: {(1, 3), (1, 1)}
Roomba is moving to (1, 1)

Step 4
X   X  

#wall following:
def wall_following(self):
    find nearest wall
        
     wall   |  obstacle location
     top wall: top (left, center, right)
     bottom wall: bottom (left, center, right)
     left wall: left (top, middle, bottom)
     right wall: right (top, middle, bottom)
     
    if there is an obstacle to my top/bottom center, top/bottom left, and top/bottom right:
    pick a random horizontal and move there.
    use the previous horizontal choice to influence the next horizontal choice, e.g. if previously moved left,     move left again.
    Keep moving horizontal until hit obstacle, then change direction to be vertical.
    
    If there is an obstacle to my top, middle, and bottom left/right:
    pick a random vertical position and move there
    use the previous vertical position to influence the next vertial choice, e.g. if previously moved up, then move up again
    Keep moving vertical until hit obstacle, then change direction to be horizontal.
    
    

In [87]:
obstacle_count = int(0.3*5*5)

#initialize empty array for storing indices
random_indices = np.zeros((obstacle_count, 2), dtype=int)

for i in range(obstacle_count):
    #randomly draw x and y value from range of grid size
    random_indices[i,:] = np.random.choice(range(5), size=2) 

state = -np.ones((5,5), dtype=int)  

for i in random_indices: 
    state[i[0],i[1]] = 1

state[:,0] = 0

print(state)
empty_cells = np.array(np.where(state == -1)).T
roomba_loc = empty_cells[np.random.choice(len(empty_cells))]
obstacle_cells = np.array(np.where(state == 1)).T
[tuple(i) for i in obstacle_cells]

[[ 0  1 -1 -1  1]
 [ 0 -1 -1  1 -1]
 [ 0 -1  1 -1 -1]
 [ 0 -1 -1 -1 -1]
 [ 0 -1 -1  1 -1]]


[(0, 1), (0, 4), (1, 3), (2, 2), (4, 3)]